In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Settings: show more in notebook (use with care)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 200)
pd.set_option("display.expand_frame_repr", False)

# ----------------------------
# 1) Read raw files
# ----------------------------
DATA_PATH = Path("..") / "data" / "raw" / "ipeds" / "C2024_A.csv"
HD_PATH   = Path("..") / "data" / "raw" / "ipeds" / "hd2024.csv"
CIP_PATH  = Path("..") / "data" / "raw" / "ipeds" / "CIPCode2020.csv"

df = pd.read_csv(
    DATA_PATH,
    low_memory=False,
    dtype={
        "UNITID": "string",
        "CIPCODE": "string",
        "MAJORNUM": "Int64",
        "AWLEVEL": "Int64",
        "CTOTALT": "Int64",
    },
)

hd = pd.read_csv(
    HD_PATH,
    low_memory=False,
    dtype={"UNITID": "string"},
)

cip_raw = pd.read_csv(
    CIP_PATH,
    low_memory=False,
    dtype={
        "CIPCode": "string",
        "CIPFamily": "string",
        # keep others if present; only using CIPTitle below
    },
)


In [3]:

# ----------------------------
# 2) Build CIP lookup tables
#    - Clean Excel-style ="01.00"
#    - Create CIP2 lookup
#    - Create CIP6 lookup (includes roll-ups like 12.0500; that's OK for titles)
# ----------------------------
cip_lookup = (
    cip_raw[["CIPCode", "CIPTitle"]]
    .rename(columns={"CIPCode": "cipcode", "CIPTitle": "cip_title"})
    .copy()
)

# Clean cipcode: strip whitespace and remove Excel wrapper ="..."
cip_lookup["cipcode"] = (
    cip_lookup["cipcode"]
    .astype("string")
    .str.strip()
    .str.replace(r'^="', "", regex=True)
    .str.replace(r'"$', "", regex=True)
)

cip_lookup["cip_title"] = cip_lookup["cip_title"].astype("string").str.strip()

# CIP2 = first two digits, left padded
cip_lookup["cip2"] = (
    cip_lookup["cipcode"]
    .str.split(".", n=1).str[0]
    .str.zfill(2)
)

# CIP2 titles come from rows that are exactly 2 digits, like "01"
cip2_only = cip_lookup.loc[
    cip_lookup["cipcode"].str.fullmatch(r"\d{2}", na=False)
].copy()

cip2_lookup = (
    cip2_only[["cipcode", "cip_title"]]
    .rename(columns={"cipcode": "cip2", "cip_title": "cip2_title"})
    .drop_duplicates(subset=["cip2"])
    .sort_values("cip2")
)

# CIP6 lookup: anything structurally NN.NNNN
# Note: this includes roll-ups like "12.0500". Titles are still useful.
cip6_lookup = (
    cip_lookup[["cipcode", "cip_title"]]
    .dropna()
    .drop_duplicates()
    .rename(columns={"cip_title": "cip6_title"})
)

cip6_lookup = cip6_lookup.loc[
    cip6_lookup["cipcode"].str.fullmatch(r"\d{2}\.\d{4}", na=False)
].copy()



In [4]:
# ----------------------------
# 3) Add institution names (HD)
# ----------------------------
hd_small = hd[["UNITID", "INSTNM"]].copy()

df_with_names = df.merge(
    hd_small,
    on="UNITID",
    how="left",
    validate="m:1"  # each UNITID should map to exactly one INSTNM in HD
)



In [5]:
# ----------------------------
# 4) Big Ten filter (manual list)
# ----------------------------
bigten_unitids = [
    '145637',  # UIUC
    '151351',  # Indiana
    '153658',  # Iowa
    '163286',  # Maryland
    '170976',  # Michigan
    '171100',  # Michigan State
    '174066',  # Minnesota
    '181464',  # Nebraska
    '147767',  # Northwestern
    '204796',  # Ohio State
    '214777',  # Penn State
    '243780',  # Purdue
    '186380',  # Rutgers
    '240444',  # Wisconsin
    '110662',  # UCLA
    '123961',  # USC
    '209551',  # Oregon
    '236948',  # Washington
]
bigten_set = set(bigten_unitids)



In [6]:
# ----------------------------
# 5) Build the “research” row-level dataset
# ----------------------------
research = (
    df_with_names
    .loc[df_with_names["UNITID"].isin(bigten_set),
         ["UNITID", "INSTNM", "CIPCODE", "MAJORNUM", "AWLEVEL", "CTOTALT"]]
    .rename(columns={
        "UNITID": "unitid",
        "INSTNM": "institution",
        "CIPCODE": "cipcode",
        "MAJORNUM": "major_number",
        "AWLEVEL": "award_level_code",
        "CTOTALT": "award_count_total",
    })
    .copy()
)

# Basic cleaning
research["unitid"] = research["unitid"].astype("string").str.strip()
research["institution"] = research["institution"].astype("string").str.strip()
research["cipcode"] = research["cipcode"].astype("string").str.strip()

# Ensure numeric-ish fields are numeric (nullable ints)
for col in ["major_number", "award_level_code", "award_count_total"]:
    research[col] = pd.to_numeric(research[col], errors="coerce").astype("Int64")



In [7]:
# ----------------------------
# 6) Award level labels
# ----------------------------
award_level_labels = {
    5: "Bachelors",
    7: "Masters",
    17: "Doctoral (Research/Scholarship)",
}
research["award_level_name"] = (
    research["award_level_code"]
    .map(award_level_labels)
    .astype("string")
)

research["degree_group"] = pd.Series(pd.NA, index=research.index, dtype="string")
research.loc[research["award_level_code"] == 5, "degree_group"] = "Bachelors"
research.loc[research["award_level_code"].isin([7, 17]), "degree_group"] = "Graduate"




In [8]:
# ----------------------------
# 7) CIP hierarchy fields + leaf/roll-up flags (this is the key change)
# ----------------------------

# CIP2 always derived from the first 2 digits
research["cip2"] = research["cipcode"].str.split(".", n=1).str[0].str.zfill(2)

# Structural format flag: "NN.NNNN"
research["is_cip_nn_nnnn"] = research["cipcode"].str.fullmatch(r"\d{2}\.\d{4}", na=False)

# Roll-up/general flag: ends with "00" in the last two digits (e.g., 12.0500, 14.0100, 99.0000)
research["is_cip_rollup_00"] = research["cipcode"].str.endswith("00", na=False)

# Leaf-level CIP6 flag: NN.NNNN AND not ending with "00"
# This is the definition you use to avoid hierarchical double counting when rolling to CIP2.
research["is_cip6_leaf"] = research["is_cip_nn_nnnn"] & ~research["is_cip_rollup_00"]

# Optional: CIP4 key for QA / drill-down (12.0503 -> 12.05, 12.0500 -> 12.05)
research["cip4"] = research["cipcode"].str.slice(0, 5)  # "NN.NN"



In [9]:
# ----------------------------
# 8) Join CIP titles (CIP2 required, CIP6 optional)
# ----------------------------
research_final = (
    research
    .merge(cip2_lookup, on="cip2", how="left", validate="m:1")
    .merge(cip6_lookup, on="cipcode", how="left", validate="m:1")
)

# Fill missing titles if desired (optional)
research_final["cip2_title"] = research_final["cip2_title"].fillna("Unknown CIP2").astype("string")
research_final["cip6_title"] = research_final["cip6_title"].fillna("Unknown CIP code").astype("string")



In [14]:
research_final.head(5)

,unitid,institution,cipcode,cip6_title,cip4,cip2,cip2_title,major_number,award_level_code,award_level_name,degree_group,award_count_total,is_cip_nn_nnnn,is_cip_rollup_00,is_cip6_leaf
0,110662,University of California-Los Angeles,03.0104,Environmental Science.,03.01,03,NATURAL RESOURCES AND CONSERVATION.,1,5,Bachelors,Bachelors,126,True,False,True
1,110662,University of California-Los Angeles,03.0104,Environmental Science.,03.01,03,NATURAL RESOURCES AND CONSERVATION.,1,17,Doctoral (Research/Scholarship),Graduate,4,True,False,True
2,110662,University of California-Los Angeles,04.0201,Architecture.,04.02,04,ARCHITECTURE AND RELATED SERVICES.,1,5,Bachelors,Bachelors,17,True,False,True
3,110662,University of California-Los Angeles,04.0201,Architecture.,04.02,04,ARCHITECTURE AND RELATED SERVICES.,1,7,Masters,Graduate,1,True,False,True
4,110662,University of California-Los Angeles,04.0201,Architecture.,04.02,04,ARCHITECTURE AND RELATED SERVICES.,1,17,Doctoral (Research/Scholarship),Graduate,0,True,False,True


In [ ]:
# ----------------------------
# 9) Final column order
# ----------------------------
final_cols = [
    "unitid",
    "institution",
    "cipcode",
    "cip6_title",
    "cip4",
    "cip2",
    "cip2_title",
    "major_number",
    "award_level_code",
    "award_level_name",
    "degree_group",
    "award_count_total",
    "is_cip_nn_nnnn",
    "is_cip_rollup_00",
    "is_cip6_leaf",
]
research_final = research_final[final_cols].copy()





In [15]:
# ----------------------------
# 10) Guardrails / sanity checks
# ----------------------------
assert research_final["unitid"].nunique() == 18, f"Expected 18 institutions, got {research_final['unitid'].nunique()}"

# CIP2 title should exist for all rows once join works (if you expect full coverage)
missing_cip2 = research_final["cip2_title"].eq("Unknown CIP2").sum()
print("Missing cip2_title rows:", missing_cip2)

# Quick distribution: leaf vs roll-up
print(research_final["is_cip6_leaf"].value_counts(dropna=False))
print(research_final["is_cip_rollup_00"].value_counts(dropna=False))



Missing cip2_title rows: 150
is_cip6_leaf
True     8540
False     340
Name: count, dtype: Int64
is_cip_rollup_00
False    8690
True      190
Name: count, dtype: Int64


In [17]:
# ----------------------------
# 11) Build viz-ready CIP2 totals (NO double counting)
#     This is the dataset you should use for the treemap.
# ----------------------------

# Select only major_number == 1 for this analysis
research_final_major1 = research_final.loc[research_final["major_number"] == 1].copy()

group_cols = ["unitid", "institution", "degree_group", "award_level_name", "award_level_code", "cip2", "cip2_title"]

# a) naive totals using all rows (likely inflated)
cip2_totals_all = (
    research_final_major1.groupby(group_cols)["award_count_total"]
    .sum()
    .reset_index(name="award_count_total_all_rows")
)

# b) safe totals using leaf-only rows (recommended)
cip2_totals_leaf = (
    research_final_major1.loc[research_final_major1["is_cip6_leaf"]]
    .groupby(group_cols)["award_count_total"]
    .sum()
    .reset_index(name="award_count_total_leaf_only")
)

# Compare inflation
cip2_compare = (
    cip2_totals_all.merge(cip2_totals_leaf, on=group_cols, how="outer")
    .fillna({"award_count_total_all_rows": 0, "award_count_total_leaf_only": 0})
)

cip2_compare["overcount_amount"] = (
    cip2_compare["award_count_total_all_rows"] - cip2_compare["award_count_total_leaf_only"]
)
cip2_compare["overcount_pct"] = (
    cip2_compare["overcount_amount"] / cip2_compare["award_count_total_leaf_only"]
).where(cip2_compare["award_count_total_leaf_only"] != 0)



In [ ]:

cip2_totals_leaf.loc[cip2_totals_leaf["cip2"] != "99"].shape

(1307, 8)

In [23]:
inst_totals = (
    cip2_totals_leaf
    .groupby(["institution", "award_level_name"])["award_count_total_leaf_only"]
    .sum()
    .reset_index()
    .sort_values(["institution", "award_level_name"])
)

inst_totals


,institution,award_level_name,award_count_total_leaf_only
0,Indiana University-Bloomington,Bachelors,7280
1,Indiana University-Bloomington,Doctoral (Research/Scholarship),426
2,Indiana University-Bloomington,Masters,3245
3,Michigan State University,Bachelors,8701
4,Michigan State University,Doctoral (Research/Scholarship),555
5,Michigan State University,Masters,2079
6,Northwestern University,Bachelors,2153
7,Northwestern University,Doctoral (Research/Scholarship),536
8,Northwestern University,Masters,4869
9,Ohio State University-Main Campus,Bachelors,11614


In [ ]:
# ----------------------------
# 12) Write outputs
# ----------------------------
OUT_DIR = Path("..") / "data" / "ipeds" / "processed" / "ipeds"
OUT_DIR.mkdir(parents=True, exist_ok=True)

research_final.to_csv(OUT_DIR / "research_bigten_completions_2024_rowlevel.csv", index=False)
cip2_totals_leaf.to_csv(OUT_DIR / "research_bigten_completions_2024_cip2_leafonly.csv", index=False)
cip2_compare.to_csv(OUT_DIR / "research_bigten_completions_2024_cip2_compare_all_vs_leaf.csv", index=False)

print("Wrote:")
print(" -", OUT_DIR / "research_bigten_completions_2024_rowlevel.csv")
print(" -", OUT_DIR / "research_bigten_completions_2024_cip2_leafonly.csv")
print(" -", OUT_DIR / "research_bigten_completions_2024_cip2_compare_all_vs_leaf.csv")